In [1]:
from keras import layers
from keras.models import Model
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
import os
import datetime
from glob import glob
from sklearn.metrics import confusion_matrix, classification_report

import nbimporter
from DataParser import generateDf, getXYfromDf

Importing Jupyter notebook from DataParser.ipynb


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [28]:
""" Hyperparameters defined here """
base_dir = './dataset'
test_path = os.path.join(base_dir, '/test_simplified.csv')
all_train_paths = glob(os.path.join(base_dir, 'train_simplified', '*.csv'))
cols = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word']
# WaveNet parameters
n_filters = 64
kernel_size = 2
dilation_depth = 8
pool_size_1 = 4
pool_size_2 = 8
batch_size = 2048
activation = 'softmax'

In [6]:
train_df, valid_df, test_df, word_encoder = generateDf(n_train=750, n_valid=75, n_test=50, n_strokes=196, path=all_train_paths)
x_train, y_train = getXYfromDf(train_df, word_encoder)
x_valid, y_valid = getXYfromDf(valid_df, word_encoder)
x_test, y_test = getXYfromDf(test_df, word_encoder)

In [7]:
def residual_block(x, i):
    tanh_out = layers.Conv1D(n_filters, 
                      kernel_size, 
                      dilation_rate = kernel_size**i, 
                      padding='causal', 
                      name='dilated_conv_%d_tanh' % (kernel_size ** i), 
                      activation='tanh'
                      )(x)
    sigm_out = layers.Conv1D(n_filters, 
                      kernel_size, 
                      dilation_rate = kernel_size**i, 
                      padding='causal', 
                      name='dilated_conv_%d_sigm' % (kernel_size ** i), 
                      activation='sigmoid'
                      )(x)
    z = layers.Multiply(name='gated_activation_%d' % (i))([tanh_out, sigm_out])
    skip = layers.Conv1D(n_filters, 1, name='skip_%d'%(i))(z)
    res = layers.Add(name='residual_block_%d' % (i))([skip, x])
    return res, skip

In [24]:
def WaveNet(inputShape, outputShape):
    stroke_input = layers.Input(shape=inputShape, name='featureInput')
    x = layers.Conv1D(n_filters, kernel_size, dilation_rate=1, padding='causal',
                      name='dilated_conv_1')(stroke_input)
    skip_connections = []
    for i in range(1, dilation_depth + 1):
        x, skip = residual_block(x, i)
        skip_connections.append(skip)
    x = layers.Add(name='skip_connections')(skip_connections)
    x = layers.Activation('relu')(x)
#     x = layers.LeakyReLU(alpha=0.1)(x)
    
    x = layers.Conv1D(n_filters, pool_size_1, strides=1, padding='same',
                     name='conv_5ms', activation='relu')(x)
    x = layers.Conv1D(output_shape[0], pool_size_2, padding='same', activation='relu',
                    name='conv_500ms')(x)
    x = layers.Conv1D(output_shape[0], pool_size_2, padding='same', activation='relu', 
                      name='conv_500ms_target_shape')(x)
    x = layers.AveragePooling1D(pool_size_2, padding='same',name = 'downsample_to_2Hz')(x)
    x = layers.Conv1D(output_shape[0], (int) (input_shape[0] / (pool_size_1*pool_size_2)), 
                      padding='same', name='final_conv')(x)
    x = layers.GlobalAveragePooling1D(name='final_pooling')(x)
    x = layers.Activation(activation, name='final_activation')(x)
    
    model = Model(input=stroke_input, output=x)
    print(model.summary())
    return model

In [25]:
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]

In [26]:
model = WaveNet(input_shape, output_shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
featureInput (InputLayer)       (None, 196, 3)       0                                            
__________________________________________________________________________________________________
dilated_conv_1 (Conv1D)         (None, 196, 64)      448         featureInput[0][0]               
__________________________________________________________________________________________________
dilated_conv_2_tanh (Conv1D)    (None, 196, 64)      8256        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
dilated_conv_2_sigm (Conv1D)    (None, 196, 64)      8256        dilated_conv_1[0][0]             
__________________________________________________________________________________________________
gated_acti

/home/jianing_sun/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fi..., inputs=Tensor("fe...)`


In [11]:
def top_3_accuracy(x,y): 
    return top_k_categorical_accuracy(x,y, 3)

In [29]:
def train(model):
    date = datetime.datetime.today().strftime('%H_%M_%m_%d')
    weight_save_path = './model/stroke_wn_%s' % date + '.h5'
    
    checkpoint = ModelCheckpoint(weight_save_path, monitor='val_loss',
                                verbose=1, save_best_only=True, period=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, 
                                  patience=1, min_lr=1e-6, mode='auto')
    early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=5)
    callback = [checkpoint, early_stop, reduce_lr]
    optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', top_3_accuracy])
    model.fit(x_train, y_train, 
              validation_data=(x_valid, y_valid),
              batch_size=batch_size,
              epochs=50,
              callbacks=callback)

In [20]:
def evaluate(model, weight_path):
    model = model.load_weights(weight_path)
    result = model.evaluate(x_test, y_test, batch_size=4096)
    print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

In [21]:
def sklearnReport(model, weight_path):
    model = model.load_weights(weight_path)
    test_cat = np.argmax(y_test, 1)
    pred_y = model.predict(x_test, batch_size = 4096)
    pred_cat = np.argmax(pred_y, 1)
    plt.matshow(confusion_matrix(test_cat, pred_cat))
    print(classification_report(test_cat, pred_cat, 
                            target_names = [x for x in word_encoder.classes_]))

In [22]:
train(model)

Train on 255000 samples, validate on 25500 samples
Epoch 1/50
255000/255000 [==============================] - 136s 534us/step - loss: 5.6313 - acc: 0.0096 - top_3_accuracy: 0.0276 - val_loss: 5.3416 - val_acc: 0.0235 - val_top_3_accuracy: 0.0578

Epoch 00001: val_loss improved from inf to 5.34156, saving model to ./model/stroke_wn_18_32_10_06.h5
Epoch 2/50
255000/255000 [==============================] - 115s 449us/step - loss: 5.2100 - acc: 0.0308 - top_3_accuracy: 0.0737 - val_loss: 5.0912 - val_acc: 0.0472 - val_top_3_accuracy: 0.1025

Epoch 00002: val_loss improved from 5.34156 to 5.09117, saving model to ./model/stroke_wn_18_32_10_06.h5
Epoch 3/50
255000/255000 [==============================] - 115s 450us/step - loss: 4.9312 - acc: 0.0594 - top_3_accuracy: 0.1314 - val_loss: 4.7714 - val_acc: 0.0734 - val_top_3_accuracy: 0.1577

Epoch 00003: val_loss improved from 5.09117 to 4.77138, saving model to ./model/stroke_wn_18_32_10_06.h5
Epoch 4/50
255000/255000 [=====================

255000/255000 [==============================] - 115s 450us/step - loss: 2.7946 - acc: 0.3766 - top_3_accuracy: 0.5769 - val_loss: 2.8765 - val_acc: 0.3649 - val_top_3_accuracy: 0.5612

Epoch 00028: val_loss improved from 2.91359 to 2.87650, saving model to ./model/stroke_wn_18_32_10_06.h5
Epoch 29/50
255000/255000 [==============================] - 115s 450us/step - loss: 2.7677 - acc: 0.3825 - top_3_accuracy: 0.5817 - val_loss: 2.8356 - val_acc: 0.3755 - val_top_3_accuracy: 0.5698

Epoch 00029: val_loss improved from 2.87650 to 2.83560, saving model to ./model/stroke_wn_18_32_10_06.h5
Epoch 30/50
255000/255000 [==============================] - 115s 449us/step - loss: 2.7417 - acc: 0.3881 - top_3_accuracy: 0.5874 - val_loss: 2.8150 - val_acc: 0.3771 - val_top_3_accuracy: 0.5747

Epoch 00030: val_loss improved from 2.83560 to 2.81499, saving model to ./model/stroke_wn_18_32_10_06.h5
Epoch 31/50
255000/255000 [==============================] - 114s 448us/step - loss: 2.7168 - acc: 0.39

In [23]:
# test 
weight_path = './model/stroke_wn_18_32_10_06.h5'
model.load_weights(weight_path)
wn_results = model.evaluate(x_test, y_test, batch_size = 4096)
print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*wn_results[1], 100*wn_results[2]))

17000/17000 [==============================] - 2s 102us/step
Accuracy: 44.1%, Top 3 Accuracy 64.2%


In [30]:
# fine-tuning
model.load_weights(weight_path)
train(model)

Train on 255000 samples, validate on 25500 samples
Epoch 1/50
255000/255000 [==============================] - 115s 452us/step - loss: 2.4132 - acc: 0.4498 - top_3_accuracy: 0.6522 - val_loss: 2.4980 - val_acc: 0.4368 - val_top_3_accuracy: 0.6364

Epoch 00001: val_loss improved from inf to 2.49796, saving model to ./model/stroke_wn_20_11_10_06.h5
Epoch 2/50
255000/255000 [==============================] - 107s 420us/step - loss: 2.3771 - acc: 0.4578 - top_3_accuracy: 0.6604 - val_loss: 2.4794 - val_acc: 0.4402 - val_top_3_accuracy: 0.6437

Epoch 00002: val_loss improved from 2.49796 to 2.47941, saving model to ./model/stroke_wn_20_11_10_06.h5
Epoch 3/50
255000/255000 [==============================] - 107s 421us/step - loss: 2.3638 - acc: 0.4594 - top_3_accuracy: 0.6624 - val_loss: 2.4665 - val_acc: 0.4434 - val_top_3_accuracy: 0.6435

Epoch 00003: val_loss improved from 2.47941 to 2.46653, saving model to ./model/stroke_wn_20_11_10_06.h5
Epoch 4/50
255000/255000 [=====================


Epoch 00028: val_loss improved from 2.24298 to 2.23995, saving model to ./model/stroke_wn_20_11_10_06.h5
Epoch 29/50
255000/255000 [==============================] - 107s 420us/step - loss: 2.0747 - acc: 0.5210 - top_3_accuracy: 0.7162 - val_loss: 2.2409 - val_acc: 0.4897 - val_top_3_accuracy: 0.6901

Epoch 00029: val_loss did not improve from 2.23995
Epoch 30/50
255000/255000 [==============================] - 107s 420us/step - loss: 2.0641 - acc: 0.5233 - top_3_accuracy: 0.7191 - val_loss: 2.2312 - val_acc: 0.4933 - val_top_3_accuracy: 0.6915

Epoch 00030: val_loss improved from 2.23995 to 2.23121, saving model to ./model/stroke_wn_20_11_10_06.h5
Epoch 31/50
255000/255000 [==============================] - 107s 421us/step - loss: 2.0607 - acc: 0.5241 - top_3_accuracy: 0.7191 - val_loss: 2.2308 - val_acc: 0.4938 - val_top_3_accuracy: 0.6927

Epoch 00031: val_loss improved from 2.23121 to 2.23082, saving model to ./model/stroke_wn_20_11_10_06.h5
Epoch 32/50
255000/255000 [============

KeyboardInterrupt: 